In [37]:
import pandas as pd
from PIL import Image,ImageDraw,ImageFont
import os
import numpy as np
import math

#### train matrix

In [38]:
test_matrix = pd.read_csv("193398_test_2014-04-01.csv")

In [39]:
test_matrix.sort_values(by='score',ascending=False,inplace=True)

In [40]:
test_matrix.reset_index(drop=True,inplace=True)

## Binning

In [41]:
def binticks(matrix,feat):
    tmp = matrix.loc[:,feat]
    tmpmin = min(tmp)
    tmpmax = max(tmp)
    tmprange = tmpmax - tmpmin
    step = tmprange / 100
    return np.arange(tmpmin,tmpmax+step,step)

In [42]:
def labelticks(matrix,feat):
    tmp = matrix.loc[:,feat]
    tmpmin = min(tmp)
    tmpmax = max(tmp)
    tmprange = tmpmax - tmpmin
    step = tmprange / 20
    return np.arange(tmpmin,tmpmax+step,step)

In [43]:
def minmax(matrix,feat):
    tmp = matrix.loc[:,feat]
    tmpmin = min(tmp)
    tmpmax = max(tmp)
    return tmpmin,tmpmax

### Axis labels

In [44]:
def addlabels(canvas,ticks):
    width = canvas.width
    height = canvas.height
    margin = 28
    newheight = height + margin
    
    unicanvas = Image.new('RGB',(width,newheight),(255,255,255))
    unicanvas.paste(canvas,(0,0))
    
    font = ImageFont.truetype('VeraMono.ttf', rectheight - 8 )
    
    # fontWidth, fontHeight = font.getsize(high) # can use if I have sizing issues

    draw = ImageDraw.Draw(unicanvas,'RGB')
    
    for i in range(len(ticks)):
        xpos = i * rectwidth * 5
        draw.text((xpos,newheight - 28 ),text=str(int(ticks[i]*100)),font=font,fill=(0,0,0))
    
    return unicanvas

## Plot

In [45]:
def overhist_label(X,feat,rectwidth,rectheight,pad):
    
    binned = pd.cut(X[feat],bins=binticks(X,feat),labels=False,include_lowest=True)
    binmax = binned.value_counts().max()
    
    px_w = len(binticks(X,feat)) * rectwidth
    px_h = binmax * rectheight
    
    font = ImageFont.truetype('VeraMono.ttf', rectheight / 2 - 4)
    
    canvas = Image.new('RGB',(px_w,px_h),(255,255,255))
    draw = ImageDraw.Draw(canvas,'RGBA')
    
    for binn in binned.value_counts().keys():
        xpos = binn * rectwidth
        ypos = px_h - rectheight

        tmp = binned[binned==binn]

        for i in tmp.index:
            bbox = [(xpos+pad,ypos+pad-1),(xpos+rectwidth-pad,ypos+rectheight-pad-1)]
            outcome = test_matrix.outcome.loc[i]
            
            if outcome==1:
                fillcolor = (220,101,113,127) # red
                if i <= 100:
                    fillcolor = (170,39,52,190) # darker red if model says '1'
            else:
                fillcolor = (112,159,210,127) # blue
                if i <= 100:
                    fillcolor = (50,103,159,190) # darker blue if model says '1'
            
            #draw.rectangle(bbox,fill=fillcolor,outline=(189,189,189,255))
            #draw.rectangle(bbox,fill=fillcolor,outline=(255,255,255,255))
            draw.rectangle(bbox,fill=fillcolor,outline=None)
            #draw.text((xpos+2,ypos+5),text=str(i),font=font,fill=(255,255,255,255))
            ypos = ypos - rectheight
    
    # range labels
    canvas = addlabels(canvas,labelticks(X,feat))
    
    return canvas

In [46]:
cfeat = 'score'
pad = 1

In [47]:
rectwidth = 28
rectheight = 28

In [48]:
im = overhist_label(test_matrix,cfeat,rectwidth,rectheight,pad)
im.save("/Users/damoncrockett/Desktop/response_view.png")